# Figure of 8 to be followed

In [1]:
# 1. Run on prompt
# daniel@simbox /opt/carla-simulator $ ./CarlaUE4.sh 

# 2. boilerplate code
import carla 
import math 
import random 
import time 
import carla_helpers as helpers

# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() 

# load Town04 map
world = client.load_world('Town04')

## Helper functions

In [2]:
def get_figure8_waypoints(world):
    road_sequence = [42, 267, 43, 35, 861, 36, 760, 37, 1602, 38, 1091, 39, 1184, 40, 1401, \
                                                                       41, 6, 45, 145, 46, 1072, 47, 774, 48, 901, 49, 1173, 50]
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(1.0)
    
    all_waypoints = []
    for road_id in road_sequence:
        road_waypoints = [wp for wp in waypoints if wp.road_id == road_id and wp.lane_id == -3]
        road_waypoints.sort(key=lambda x: x.s)
        all_waypoints.extend(road_waypoints)
        
    return all_waypoints

def set_spectator_camera_following_car(world, vehicle):
    """
    Position the spectator 3 meters above and 5 meters behind the vehicle.
    
    Parameters:
    - world: CARLA world object.
    - vehicle: CARLA vehicle object.
    
    Returns:
    - spectator: CARLA spectator object.
    """
    spectator = world.get_spectator()
    vehicle_transform = vehicle.get_transform()

    # Get the vehicle's forward vector (direction it's facing)
    forward_vector = vehicle_transform.get_forward_vector()

    # Calculate the offset position: 5 meters behind and 3 meters above the vehicle
    offset_location = vehicle_transform.location + carla.Location(
        x=-5 * forward_vector.x,  # 5 meters behind along the forward axis
        y=-5 * forward_vector.y,  # 5 meters behind along the forward axis
        z=3  # 3 meters above
    )

    # Set the spectator transform with a fixed pitch for a better view
    spectator.set_transform(carla.Transform(
        offset_location,
        carla.Rotation(pitch=-15, yaw=vehicle_transform.rotation.yaw, roll=0)  # Slightly tilted down
    ))
    return spectator

def compute_control(vehicle, target_wp, target_speed=10):  # Reduced target speed to 10 km/h
    """
    Compute vehicle control to reach the target waypoint at a specified speed.
    
    Parameters:
    - vehicle: CARLA vehicle object.
    - target_wp: Target waypoint.
    - target_speed: Desired speed in km/h (default is 10 km/h).
    
    Returns:
    - control: CARLA VehicleControl object.
    """
    control = carla.VehicleControl()

    # Get current state
    current_transform = vehicle.get_transform()
    current_velocity = vehicle.get_velocity()
    speed = 3.6 * math.sqrt(current_velocity.x**2 + current_velocity.y**2)  # Convert to km/h

    # Steering control
    forward = current_transform.get_forward_vector()
    target_vector = target_wp.transform.location - current_transform.location
    forward_dot = forward.x * target_vector.x + forward.y * target_vector.y
    right_dot = -forward.y * target_vector.x + forward.x * target_vector.y
    steering = math.atan2(right_dot, forward_dot) / math.pi
    control.steer = max(-1.0, min(1.0, steering))  # Clamp steering to [-1, 1]

    # Speed control
    speed_error = target_speed - speed
    if speed_error > 0:
        # If below target speed, apply throttle
        control.throttle = min(0.3, speed_error / target_speed)  # Reduced throttle limit to 0.3
        control.brake = 0.0
    else:
        # If above target speed, apply brake
        control.throttle = 0.0
        control.brake = min(0.3, -speed_error / target_speed)  # Reduced brake limit to 0.3

    return control

def drive_figure_eight(world, vehicle, waypoints, target_speed=10):
    """
    Drive the vehicle along the figure-8 track at a specified speed.
    
    Parameters:
    - world: CARLA world object.
    - vehicle: CARLA vehicle object.
    - waypoints: List of waypoints for the figure-8 track.
    - target_speed: Desired speed in km/h (default is 10 km/h).
    """
    try:
        for i, wp in enumerate(waypoints):
            print(f"Current target waypoint {i + 1}/{len(waypoints)}: {wp.transform.location}")
            
            # Draw the current target waypoint
            world.debug.draw_point(
                wp.transform.location,
                size=0.2,  # Size of the marker
                color=carla.Color(255, 0, 0),  # Red color
                life_time=5.0  # Persist for 5 seconds
            )
            
            # Drive toward the current waypoint
            while True:
                control = compute_control(vehicle, wp, target_speed)
                vehicle.apply_control(control)
                
                # Update spectator camera
                set_spectator_camera_following_car(world, vehicle)
                
                # Check if the vehicle is close to the current waypoint
                current_location = vehicle.get_transform().location
                distance_to_waypoint = current_location.distance(wp.transform.location)
                if distance_to_waypoint < 2.0:  # Proceed to the next waypoint if within 2 meters
                    break
                
                if world.get_settings().synchronous_mode:
                    world.tick()
                else:
                    time.sleep(0.1)  # Increased delay to 0.1 seconds
                    
    except KeyboardInterrupt:
        print("Simulation interrupted by user.")
        control = carla.VehicleControl(throttle=0, brake=1)
        vehicle.apply_control(control)

def draw_permanent_waypoint_lines(world, waypoints, color=carla.Color(0, 255, 0), thickness=0.02):
    """
    Draw permanent lines linking every waypoint on the road.
    
    Parameters:
    - world: CARLA world object.
    - waypoints: List of waypoints to link.
    - color: Color of the lines (default is neon green).
    - thickness: Thickness of the lines (default is 0.1 meters).
    """
    for i in range(len(waypoints) - 1):
        # Get the current and next waypoint
        wp1 = waypoints[i]
        wp2 = waypoints[i + 1]
        
        # Draw a line between the two waypoints
        world.debug.draw_line(
            wp1.transform.location,  # Start point
            wp2.transform.location,  # End point
            thickness=thickness,     # Line thickness
            color=color,             # Line color
            life_time=0             # Permanent line (life_time=0 means infinite)
        )        

## Instantiate vehicle and place on start of figure of 8

In [3]:
def main():
    # Connect to the CARLA server
    client = carla.Client("localhost", 2000)
    client.set_timeout(10.0)
    world = client.get_world()

    # Enable synchronous mode
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.05
    world.apply_settings(settings)

    try:
        # Get waypoints for the figure-8 track
        waypoints = get_figure8_waypoints(world)
        print(f"Number of waypoints retrieved: {len(waypoints)}")

        # Draw permanent lines linking the waypoints
        draw_permanent_waypoint_lines(world, waypoints, color=carla.Color(0, 255, 255))  # Neon cyan

        # Spawn vehicle at the first waypoint
        blueprint_library = world.get_blueprint_library()
        vehicle_bp = blueprint_library.filter("vehicle.tesla.model3")[0]  # Choose a vehicle blueprint
        spawn_point = waypoints[0].transform
        spawn_point.location.z += 2.0  # Adjust spawn height to avoid collisions
        print(f"Vehicle spawn point: {spawn_point.location}")

        try:
            vehicle = world.spawn_actor(vehicle_bp, spawn_point)
            print(f"Vehicle spawned at {spawn_point.location}")
        except Exception as e:
            print(f"Failed to spawn vehicle: {e}")
            return  # Exit the script if spawning fails

        # Drive the vehicle around the figure-8 track at 10 km/h
        try:
            drive_figure_eight(world, vehicle, waypoints, target_speed=10)
        except Exception as e:
            print(f"Error during driving: {e}")
    except KeyboardInterrupt:
        print("Simulation interrupted by user.")
    finally:
        # Clean up
        if 'vehicle' in locals():
            vehicle.destroy()
            print("Vehicle destroyed.")

main()

Number of waypoints retrieved: 3063
Vehicle spawn point: Location(x=160.342804, y=-368.140472, z=2.000000)
Vehicle spawned at Location(x=160.342804, y=-368.140472, z=2.000000)
Current target waypoint 1/3063: Location(x=160.342804, y=-368.140472, z=0.000000)
Current target waypoint 2/3063: Location(x=161.342758, y=-368.131714, z=0.000000)
Current target waypoint 3/3063: Location(x=162.342728, y=-368.122955, z=0.000000)
Current target waypoint 4/3063: Location(x=163.342682, y=-368.114227, z=0.000000)
Current target waypoint 5/3063: Location(x=164.342651, y=-368.105469, z=0.000000)
Current target waypoint 6/3063: Location(x=165.342606, y=-368.096710, z=0.000000)
Current target waypoint 7/3063: Location(x=166.342575, y=-368.087952, z=0.000000)
Current target waypoint 8/3063: Location(x=167.342529, y=-368.079224, z=0.000000)
Current target waypoint 9/3063: Location(x=168.342499, y=-368.070465, z=0.000000)
Current target waypoint 10/3063: Location(x=169.342453, y=-368.061707, z=0.000000)
Cur